In [4]:
import pandas as pd

In [5]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as st
from sklearn import metrics

class DelongTest():
    def __init__(self,preds1,preds2,label,threshold=0.05):
        '''
        preds1:the output of model1
        preds2:the output of model2
        label :the actual label
        '''
        self._preds1=preds1
        self._preds2=preds2
        self._label=label
        self.threshold=threshold
        #self._show_result()

    def _auc(self,X, Y)->float:
        return 1/(len(X)*len(Y)) * sum([self._kernel(x, y) for x in X for y in Y])

    def _kernel(self,X, Y)->float:
        '''
        Mann-Whitney statistic
        '''
        return .5 if Y==X else int(Y < X)

    def _structural_components(self,X, Y)->list:
        V10 = [1/len(Y) * sum([self._kernel(x, y) for y in Y]) for x in X]
        V01 = [1/len(X) * sum([self._kernel(x, y) for x in X]) for y in Y]
        return V10, V01

    def _get_S_entry(self,V_A, V_B, auc_A, auc_B)->float:
        return 1/(len(V_A)-1) * sum([(a-auc_A)*(b-auc_B) for a,b in zip(V_A, V_B)])
    
    def _z_score(self,var_A, var_B, covar_AB, auc_A, auc_B):
        return (auc_A - auc_B)/((var_A + var_B - 2*covar_AB )**(.5)+ 1e-8)

    def _group_preds_by_label(self,preds, actual)->list:
        X = [p for (p, a) in zip(preds, actual) if a]
        Y = [p for (p, a) in zip(preds, actual) if not a]
        return X, Y

    def _compute_z_p(self):
        X_A, Y_A = self._group_preds_by_label(self._preds1, self._label)
        X_B, Y_B = self._group_preds_by_label(self._preds2, self._label)

        V_A10, V_A01 = self._structural_components(X_A, Y_A)
        V_B10, V_B01 = self._structural_components(X_B, Y_B)

        auc_A = self._auc(X_A, Y_A)
        auc_B = self._auc(X_B, Y_B)

        # Compute entries of covariance matrix S (covar_AB = covar_BA)
        var_A = (self._get_S_entry(V_A10, V_A10, auc_A, auc_A) * 1/len(V_A10)+ self._get_S_entry(V_A01, V_A01, auc_A, auc_A) * 1/len(V_A01))
        var_B = (self._get_S_entry(V_B10, V_B10, auc_B, auc_B) * 1/len(V_B10)+ self._get_S_entry(V_B01, V_B01, auc_B, auc_B) * 1/len(V_B01))
        covar_AB = (self._get_S_entry(V_A10, V_B10, auc_A, auc_B) * 1/len(V_A10)+ self._get_S_entry(V_A01, V_B01, auc_A, auc_B) * 1/len(V_A01))

        # Two tailed test
        z = self._z_score(var_A, var_B, covar_AB, auc_A, auc_B)
        p = st.norm.sf(abs(z))*2

        return z,p

    def _show_result(self):
        z,p=self._compute_z_p()
        print(f"z score = {z:.5f};\np value = {p:.5f};")
        if p < self.threshold :
            print("There is a significant difference")
        else:
            print("There is NO significant difference")


In [12]:
cliradio_train = pd.read_csv('./out_data/clinical_radio_train.csv')
cliradio_test = pd.read_csv('./out_data/clinical_radio_test.csv')
cliradio_extest = pd.read_csv('./out_data/clinical_radio_extest.csv')
cliradio_extest.head()

,y_true,y_pred,y_score
0,1,False,0.651646
1,0,False,0.028857
2,0,False,0.278621
3,1,True,0.920497
4,1,True,0.969395


In [8]:
cli_train = pd.read_csv('./out_data/clinical_train.csv')
cli_test = pd.read_csv('./out_data/clinical_test.csv')
cli_extest = pd.read_csv('./out_data/clinical_extest.csv')

In [27]:
radio = pd.read_csv('../Train/out_data/nomogram_feas_radiomics.csv')
radio_train = radio[radio['dataset']=='training']
radio_test = radio[radio['dataset']=='test']
radio_extest = radio[radio['dataset']=='external']
radio.head(2)
radio['dataset'].value_counts()

training    158
test         70
external     45
Name: dataset, dtype: int64

In [28]:
# radio
radio_train_score = radio_train['radiomics_score']
radio_test_score = radio_test['radiomics_score']
radio_extest_score = radio_extest['radiomics_score']

# clinical
cli_train_score = cli_train['y_score']
cli_test_score = cli_test['y_score']
cli_extest_score = cli_extest['y_score']

# clinical
cliradio_train_score = cliradio_train['y_score']
cliradio_test_score = cliradio_test['y_score']
cliradio_extest_score = cliradio_extest['y_score']

# y_true
cliradio_train_true = cliradio_train['y_true']
cliradio_test_true = cliradio_test['y_true']
cliradio_extest_true = cliradio_extest['y_true']

In [30]:
# test set
score_list = [radio_test_score,
              cli_test_score,
              cliradio_test_score
             ]
score_list
y_true = cliradio_test_true


for i, s1 in enumerate(score_list):
    for j, s2 in enumerate(score_list):
        print(f'{i}-{j} p-value = {DelongTest(s1, s2, y_true)._compute_z_p()[1]}')

0-0 p-value = 1.0
0-1 p-value = 0.011169848503998393
0-2 p-value = 0.0020452839181291055
1-0 p-value = 0.011169848503998393
1-1 p-value = 1.0
1-2 p-value = 0.8348067363896076
2-0 p-value = 0.0020452839181291055
2-1 p-value = 0.8348067363896076
2-2 p-value = 1.0


In [31]:
# extest set
score_list = [radio_extest_score,
              cli_extest_score,
              cliradio_extest_score
             ]
score_list
y_true = cliradio_extest_true
print(len(y_true))
print(len(radio_extest_score))
print(len(cli_extest_score))
print(len(cliradio_extest_score))

for i, s1 in enumerate(score_list):
    for j, s2 in enumerate(score_list):
        print(f'{i}-{j} p-value = {DelongTest(s1, s2, y_true)._compute_z_p()[1]}')

45
45
45
45
0-0 p-value = 1.0
0-1 p-value = 0.38187091753362723
0-2 p-value = 0.057496418488634
1-0 p-value = 0.38187091753362723
1-1 p-value = 1.0
1-2 p-value = 0.5267146341221188
2-0 p-value = 0.057496418488634
2-1 p-value = 0.5267146341221188
2-2 p-value = 1.0


In [35]:
list(cliradio_extest)

['y_true', 'y_pred', 'y_score']

In [36]:
# extest set
score_list = [list(radio_test_score)+list(radio_extest_score),
              list(cli_test_score)+list(cli_extest_score),
              list(cliradio_extest_score)+list(cliradio_extest_score)
             ]
score_list
y_true = list(cliradio_test_true)+list(cliradio_extest_true)
print(len(y_true))
print(len(radio_extest_score))
print(len(cli_extest_score))
print(len(cliradio_extest_score))

for i, s1 in enumerate(score_list):
    for j, s2 in enumerate(score_list):
        print(f'{i}-{j} p-value = {DelongTest(s1, s2, y_true)._compute_z_p()[1]}')

115
45
45
45
0-0 p-value = 1.0
0-1 p-value = 0.007195977393302671
0-2 p-value = 0.004310628523995916
1-0 p-value = 0.007195977393302671
1-1 p-value = 1.0
1-2 p-value = 9.547577206167866e-08
2-0 p-value = 0.005497621973929766
2-1 p-value = 7.256287901700801e-08
2-2 p-value = 1.0


In [37]:
a = [1,2,3,4]
a = a[:2]
a

[1, 2]

In [41]:
import numpy as np
n1 = np.ones([3,3,3])
-2048*np.ones(n1.shape)

array([[[-2048., -2048., -2048.],
        [-2048., -2048., -2048.],
        [-2048., -2048., -2048.]],

       [[-2048., -2048., -2048.],
        [-2048., -2048., -2048.],
        [-2048., -2048., -2048.]],

       [[-2048., -2048., -2048.],
        [-2048., -2048., -2048.],
        [-2048., -2048., -2048.]]])